# Introduction to DataFrames
**[Bogumił Kamiński](http://bogumilkaminski.pl/about/), January 27, 2019**

In [1]:
using DataFrames # load package

## Загрузка и сохранение DataFrames

[CSV](https://github.com/JuliaData/CSV.jl)

[Whats the difference between csv and csvfiles](https://discourse.julialang.org/t/whats-the-difference-between-csv-jl-and-csvfiles-jl/17950/2)

[JLD2](https://github.com/JuliaIO/JLD2.jl)

[Feather](https://github.com/JuliaData/Feather.jl)

[Serialization](https://github.com/JuliaStdlibs/Serialization.jl)

Здесь мы загрузим `CSV` и` CSVFiles` для чтения и записи файлов CSV, а также `JLD2`,` Feather` и `Serialization`, которые позволяют нам работать с двоичным форматом.

In [ ]:
using Pkg
Pkg.add("CSV")
Pkg.add("CSVFiles")
Pkg.add("JLD2")
Pkg.add("Serialization")
Pkg.add("Feather")

In [2]:
using CSV
using CSVFiles
using JLD2
using Serialization
using Feather

Давайте создадим простой DataFrame для тестирования,

In [3]:
x = DataFrame(A=[true, false, true], B=[1, 2, missing],
              C=[missing, "b", "c"], D=['a', missing, 'c'])


3×4 DataFrame
│ Row │ A     │ B       │ C       │ D       │
│     │ Bool  │ Int64⍰  │ String⍰ │ Char⍰   │
├─────┼───────┼─────────┼─────────┼─────────┤
│ 1   │ true  │ 1       │ missing │ 'a'     │
│ 2   │ false │ 2       │ b       │ missing │
│ 3   │ true  │ missing │ c       │ 'c'     │

и используйте `eltypes`, чтобы посмотреть на типы столбцов.

In [4]:
eltypes(x)

4-element Array{Type,1}:
 Bool                  
 Union{Missing, Int64} 
 Union{Missing, String}
 Union{Missing, Char}  

### CSV.jl

Давайте использовать `CSV` для сохранения` x` на диск; убедитесь, что `x1.csv` не конфликтует с каким-либо файлом в вашем рабочем каталоге.

In [5]:
CSV.write("x1.csv", x)

"x1.csv"

Теперь мы можем увидеть, как это было сохранено, прочитав `x.csv`.

In [6]:
print(read("x1.csv", String))

A,B,C,D
true,1,,a
false,2,b,
true,,c,c


Мы также можем загрузить его обратно (`use_mmap = false` отключает отображение памяти, так что в Windows файл может быть удален в том же сеансе, в других ОС он не нужен).

In [7]:
y = CSV.read("x1.csv", use_mmap=false)

3×4 DataFrame
│ Row │ A     │ B       │ C       │ D       │
│     │ Bool⍰ │ Int64⍰  │ String⍰ │ String⍰ │
├─────┼───────┼─────────┼─────────┼─────────┤
│ 1   │ true  │ 1       │ missing │ a       │
│ 2   │ false │ 2       │ b       │ missing │
│ 3   │ true  │ missing │ c       │ c       │

При загрузке в `DataFrame` из` CSV`, все столбцы разрешают `Missing` по умолчанию. Обратите внимание, что типы столбцов изменились!

In [8]:
eltypes(y)

4-element Array{Union,1}:
 Union{Missing, Bool}  
 Union{Missing, Int64} 
 Union{Missing, String}
 Union{Missing, String}

### CSVFiles.jl

Теперь мы будем использовать `CSVFiles` для достижения того же. Сначала мы сохраняем файл. Обратите внимание, что мы переопределяем по умолчанию `nastring`, то есть` 'NA' `, потому что у нас есть пропуски в нечисловых столбцах.

In [9]:
x |> save("x2.csv", nastring="")

и посмотреть сохраненный файл:

In [10]:
print(read("x2.csv", String))

"A","B","C","D"
true,1,,a
false,2,"b",
true,,"c",c


Мы можем загрузить его обратно, используя `load`:

In [11]:
y = load("x2.csv") |> DataFrame

3×4 DataFrame
│ Row │ A      │ B       │ C      │ D      │
│     │ String │ Int64⍰  │ String │ String │
├─────┼────────┼─────────┼────────┼────────┤
│ 1   │ true   │ 1       │        │ a      │
│ 2   │ false  │ 2       │ b      │        │
│ 3   │ true   │ missing │ c      │ c      │

Давайте еще раз проверим типы элементов:

In [12]:
eltypes(y)

4-element Array{Type,1}:
 String               
 Union{Missing, Int64}
 String               
 String               

Обратите внимание, что в столбцах `:C` и `:D` пропуски считывались как пустые строки

### JLD2.jl

Теперь давайте сохраним `x` в файл в двоичном формате; убедитесь, что `x.jld` не существует в вашем рабочем каталоге.

In [13]:
@save "x.jld2" x

После загрузки в `x.jld2` как` y`, `y` идентично` x`.

In [14]:
using FileIO # необходимо для функционального интерфейса с JLD2, расширение 'jld2' обязательно для определения типа файла
y = load("x.jld2", "x")

3×4 DataFrame
│ Row │ A     │ B       │ C       │ D       │
│     │ Bool  │ Int64⍰  │ String⍰ │ Char⍰   │
├─────┼───────┼─────────┼─────────┼─────────┤
│ 1   │ true  │ 1       │ missing │ 'a'     │
│ 2   │ false │ 2       │ b       │ missing │
│ 3   │ true  │ missing │ c       │ missing │

Обратите внимание, что типы столбцов `y` совпадают с типами столбцов` x`!

In [15]:
eltypes(y)

4-element Array{Type,1}:
 Bool                  
 Union{Missing, Int64} 
 Union{Missing, String}
 Union{Missing, Char}  

### Serialization

Теперь мы используем сериализацию для сохранения `x`. Обратите внимание, что в общем случае этот процесс не будет работать, если чтение и запись выполняются разными версиями Julia или экземпляром Julia с другим системным образом.

In [16]:
open("x.bin", "w") do io
    serialize(io, x)
end

Теперь мы загружаем обратно сохраненный файл в переменную `y`. Снова `y` идентично` x`.

In [17]:
y = open(deserialize, "x.bin")

3×4 DataFrame
│ Row │ A     │ B       │ C       │ D       │
│     │ Bool  │ Int64⍰  │ String⍰ │ Char⍰   │
├─────┼───────┼─────────┼─────────┼─────────┤
│ 1   │ true  │ 1       │ missing │ 'a'     │
│ 2   │ false │ 2       │ b       │ missing │
│ 3   │ true  │ missing │ c       │ 'c'     │

In [18]:
eltypes(y)

4-element Array{Type,1}:
 Bool                  
 Union{Missing, Int64} 
 Union{Missing, String}
 Union{Missing, Char}  

### Feather.jl

Наконец, мы используем формат Feather, который позволяет, в частности, обмениваться данными с R или Python.

In [19]:
x.D = passmissing(string).(x.D) # Feather format does not support Char type

3-element Array{Union{Missing, String},1}:
 "a"    
 missing
 "c"    

In [20]:
Feather.write("x.feather", x)

"x.feather"

In [21]:
y = Feather.materialize("x.feather") # Feather.read is a lazy alternative

3×4 DataFrame
│ Row │ A     │ B       │ C       │ D       │
│     │ Bool  │ Int64⍰  │ String⍰ │ String⍰ │
├─────┼───────┼─────────┼─────────┼─────────┤
│ 1   │ true  │ 1       │ missing │ a       │
│ 2   │ false │ 2       │ b       │ missing │
│ 3   │ true  │ missing │ c       │ c       │

In [22]:
eltypes(y)

4-element Array{Type,1}:
 Bool                  
 Union{Missing, Int64} 
 Union{Missing, String}
 Union{Missing, String}

### Basic bechmarking

Далее мы создадим файлы `bigdf1.csv`,` bigdf2.csv` и `bigdf.jld2`, поэтому будьте осторожны, так как у вас еще нет этих файлов на диске! 

В частности, мы определим, сколько времени нам понадобится для записи `DataFrame` с 10 ^ 3 строками и 10 ^ 5 столбцами в файлы` .csv` и `.jld2`.

In [23]:
bigdf = DataFrame(Bool, 10^3, 10^2)
bigdf[1] = Int.(bigdf[1])
bigdf[2] = bigdf[2] .+ 0.5
bigdf[3] = string.(bigdf[3], ", as string")
println("First run")
@time CSV.write("bigdf1.csv", bigdf)
@time bigdf |> save("bigdf2.csv")
@time @save "bigdf.jld2" bigdf
@time open(io -> serialize(io, bigdf), "bigdf.bin", "w")
@time Feather.write("bigdf.feather", bigdf)
println("Second run")
@time CSV.write("bigdf1.csv", bigdf)
@time bigdf |> save("bigdf2.csv")
@time @save "bigdf.jld2" bigdf
@time open(io -> serialize(io, bigdf), "bigdf.bin", "w")
@time Feather.write("bigdf.feather", bigdf)
getfield.(stat.(["bigdf1.csv", "bigdf2.csv", "bigdf.jld2", "bigdf.bin", "bigdf.feather"]), :size)

First run
  0.671150 seconds (1.84 M allocations: 87.415 MiB, 6.30% gc time)
  0.474216 seconds (900.05 k allocations: 46.476 MiB, 4.19% gc time)
  0.253247 seconds (701.92 k allocations: 34.415 MiB, 7.84% gc time)
  0.109791 seconds (441.00 k allocations: 21.973 MiB, 8.78% gc time)
  0.149717 seconds (374.04 k allocations: 26.126 MiB, 7.00% gc time)
Second run
  0.017808 seconds (54.97 k allocations: 1.059 MiB)
  0.010548 seconds (5.53 k allocations: 340.094 KiB)
  0.017534 seconds (18.87 k allocations: 633.605 KiB)
  0.008291 seconds (4.81 k allocations: 243.089 KiB)
  0.024257 seconds (34.02 k allocations: 9.234 MiB, 40.43% gc time)


5-element Array{Int64,1}:
 599312
 599512
 189346
  69149
  54528

In [24]:
println("First run")
@time CSV.read("bigdf1.csv")
@time load("bigdf2.csv") |> DataFrame
@time load("bigdf.jld2", "bigdf")
@time open(deserialize, "bigdf.bin")
@time Feather.materialize("bigdf.feather")
println("Second run")
@time CSV.read("bigdf1.csv")
@time load("bigdf2.csv") |> DataFrame
@time load("bigdf.jld2", "bigdf")
@time open(deserialize, "bigdf.bin")
@time Feather.materialize("bigdf.feather");

First run
  0.306265 seconds (845.83 k allocations: 29.777 MiB, 3.71% gc time)
  0.863360 seconds (2.42 M allocations: 212.475 MiB, 18.73% gc time)
  0.139398 seconds (390.40 k allocations: 20.071 MiB, 4.83% gc time)
  0.023486 seconds (62.53 k allocations: 1.551 MiB)
  0.312782 seconds (639.75 k allocations: 32.185 MiB, 2.00% gc time)
Second run
  0.024117 seconds (52.45 k allocations: 1.117 MiB)
  0.049935 seconds (313.05 k allocations: 102.351 MiB, 49.80% gc time)
  0.003044 seconds (4.18 k allocations: 428.297 KiB)
  0.003231 seconds (49.60 k allocations: 958.953 KiB)
  0.002990 seconds (26.29 k allocations: 1.084 MiB)


Наконец, давайте почистимся. Не запускайте следующую ячейку, если вы не уверены, что она не сотрет ваши важные файлы.

In [25]:
foreach(rm, ["x1.csv", "x2.csv", "x.jld2", "x.bin", "x.feather",
             "bigdf1.csv", "bigdf2.csv", "bigdf.jld2", "bigdf.bin", "bigdf.feather"])